# **Import libraries**

In [3]:
import math
import numpy as np
import pandas as pd


# **Load CSV file** 

In [7]:
 df = pd.read_csv("storms.csv", header=0,sep=',')
 df.columns

Index(['storm_name', 'year', 'date', 'time', 'wind_power', 'air_pressure',
       'storm_type', 'lat', 'long', 'Ocean'],
      dtype='object')

In [8]:
df

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
0,ANA,2021,5/22/2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean
1,ANA,2021,5/22/2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean
2,ANA,2021,5/22/2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean
3,ANA,2021,5/22/2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean
4,ANA,2021,5/22/2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean
...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,09/03/1951,12:00:00 PM,0.0,985.0,Unknown,20.00,105.50,Western Pacific
217925,NORA,1951,09/03/1951,3:00:00 PM,0.0,987.0,Unknown,19.92,105.40,Western Pacific
217926,NORA,1951,09/03/1951,6:00:00 PM,0.0,990.0,Unknown,20.00,105.00,Western Pacific
217927,NORA,1951,09/03/1951,9:00:00 PM,0.0,991.0,Unknown,20.40,103.95,Western Pacific


# **DataFrame information** 

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 217929 entries, 0 to 217928
Data columns (total 10 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   storm_name    217929 non-null  object 
 1   year          217929 non-null  int64  
 2   date          217929 non-null  object 
 3   time          217929 non-null  object 
 4   wind_power    217822 non-null  float64
 5   air_pressure  172833 non-null  float64
 6   storm_type    217929 non-null  object 
 7   lat           217563 non-null  float64
 8   long          217563 non-null  float64
 9   Ocean         217929 non-null  object 
dtypes: float64(4), int64(1), object(5)
memory usage: 16.6+ MB


In [10]:
df.describe(include='all')

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
count,217929,217929.000000,217929,217929,217822.000000,172833.000000,217929,217563.000000,217563.000000,217929
unique,1417,NaN,14568,297,NaN,NaN,32,NaN,NaN,6
top,NOT_NAMED,NaN,08/03/2014,12:00:00 PM,NaN,NaN,Tropical Storm,NaN,NaN,Atlantic Ocean
freq,15436,NaN,81,27476,NaN,NaN,54140,NaN,NaN,55939
mean,NaN,1993.548720,NaN,NaN,46.915486,988.729513,NaN,14.700567,6.702697,NaN
std,NaN,18.983645,NaN,NaN,33.648374,20.563756,NaN,18.369363,112.778145,NaN
min,NaN,1951.000000,NaN,NaN,-1.000000,870.000000,NaN,-60.500000,-180.000000,NaN
25%,NaN,1980.000000,NaN,NaN,29.000000,980.000000,NaN,11.540000,-100.100000,NaN
50%,NaN,1996.000000,NaN,NaN,41.000000,996.000000,NaN,17.200000,-32.500000,NaN
75%,NaN,2010.000000,NaN,NaN,66.000000,1004.000000,NaN,25.200000,124.800000,NaN


# **Cleaning Data**

**1.** we removed storms without latitude and longitude coordinate.

**2.** we filled wind power and air pressure by formula that related only to Pacific Ocean.
For other oceans for wind power we took the air pressure from the same storm and calculate the meadian of wind power according to the air pressure in the same ocean, and the same for air pressure.

**3.** we removed last missing values.

**4.** we dropped duplications.

**5.** we splited 'Date' to 'year', 'Month' and 'Day'.

**6.** we added Beaufort Scale.

**7.** we changed the 'Ocean' column to categorical column.


In [11]:
# remove storms without lat and long coordinate.
df.dropna(subset=['lat', 'long'], axis=0, inplace=True)
df

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
0,ANA,2021,5/22/2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean
1,ANA,2021,5/22/2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean
2,ANA,2021,5/22/2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean
3,ANA,2021,5/22/2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean
4,ANA,2021,5/22/2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean
...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,09/03/1951,12:00:00 PM,0.0,985.0,Unknown,20.00,105.50,Western Pacific
217925,NORA,1951,09/03/1951,3:00:00 PM,0.0,987.0,Unknown,19.92,105.40,Western Pacific
217926,NORA,1951,09/03/1951,6:00:00 PM,0.0,990.0,Unknown,20.00,105.00,Western Pacific
217927,NORA,1951,09/03/1951,9:00:00 PM,0.0,991.0,Unknown,20.40,103.95,Western Pacific


In [12]:
def windSpeedToPressure(windSpeedInMPH):
    windSpeedInMS = windSpeedInMPH * 0.44704
    return round(1014.9 - 0.361451 * windSpeedInMS - 0.00259 * windSpeedInMS ** 2)


def pressureToWindSpeed(pressure):
    # Only in Pacific Ocean
    # https://sciencing.com/convert-wind-speed-pressure-5814125.html
    # convertor -https://www.metric-conversions.org/speed/miles-per-hour-to-meters-per-second.htm
    a = -0.00259
    b = -0.361451
    c = 1014.9 - pressure
    try:
        d = (b ** 2) - (4 * a * c)
        sol1 = (-b - math.sqrt(d)) / (2 * a)
        sol2 = (-b + math.sqrt(d)) / (2 * a)
        return round(max(sol1, sol2))
    except:
        return None


In [13]:

def manipulatePacific(df):
    dataframe = df.copy()
    for ind, row in dataframe.iterrows():
        if math.isnan(row['air_pressure']) and row['wind_power'] > 0:
            dataframe.loc[ind, 'air_pressure'] = float(windSpeedToPressure(row['wind_power']))
        elif row['air_pressure'] is not None and row['wind_power'] <= 0.0:
            dataframe.loc[ind, 'wind_power'] = pressureToWindSpeed(row['air_pressure'])
    return dataframe

In [14]:
def fillMissingWindOrPressure(df, columnName, compareTo):
    missingValues = df[np.logical_or(df[columnName] == 0.0, df[columnName].isnull())].index
    for ind in missingValues:
        row = df.loc[ind].copy()
        columnValues = df[np.logical_and(row['Ocean'] == df['Ocean'],
                                         row[compareTo] == df[compareTo])][columnName]
        df.loc[ind, columnName] = columnValues[columnValues != 0].median()

In [15]:
# fill air_pressure column
fillMissingWindOrPressure(df, 'air_pressure', 'wind_power')

In [16]:
# fill wind_power column
fillMissingWindOrPressure(df, 'wind_power', 'air_pressure')

In [17]:
# remove storms with wrong wind_power calculation. (outliers)
df.drop(df[df['wind_power'] <= 0].index, inplace=True)
df

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
0,ANA,2021,5/22/2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean
1,ANA,2021,5/22/2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean
2,ANA,2021,5/22/2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean
3,ANA,2021,5/22/2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean
4,ANA,2021,5/22/2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean
...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,09/03/1951,12:00:00 PM,58.0,985.0,Unknown,20.00,105.50,Western Pacific
217925,NORA,1951,09/03/1951,3:00:00 PM,54.0,987.0,Unknown,19.92,105.40,Western Pacific
217926,NORA,1951,09/03/1951,6:00:00 PM,52.0,990.0,Unknown,20.00,105.00,Western Pacific
217927,NORA,1951,09/03/1951,9:00:00 PM,48.0,991.0,Unknown,20.40,103.95,Western Pacific


In [18]:
# remove last missing values
df.dropna(axis=0, inplace=True)
df

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
0,ANA,2021,5/22/2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean
1,ANA,2021,5/22/2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean
2,ANA,2021,5/22/2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean
3,ANA,2021,5/22/2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean
4,ANA,2021,5/22/2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean
...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,09/03/1951,12:00:00 PM,58.0,985.0,Unknown,20.00,105.50,Western Pacific
217925,NORA,1951,09/03/1951,3:00:00 PM,54.0,987.0,Unknown,19.92,105.40,Western Pacific
217926,NORA,1951,09/03/1951,6:00:00 PM,52.0,990.0,Unknown,20.00,105.00,Western Pacific
217927,NORA,1951,09/03/1951,9:00:00 PM,48.0,991.0,Unknown,20.40,103.95,Western Pacific


In [19]:
# drop duplicates
df.drop_duplicates()
df

,storm_name,year,date,time,wind_power,air_pressure,storm_type,lat,long,Ocean
0,ANA,2021,5/22/2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean
1,ANA,2021,5/22/2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean
2,ANA,2021,5/22/2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean
3,ANA,2021,5/22/2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean
4,ANA,2021,5/22/2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean
...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,09/03/1951,12:00:00 PM,58.0,985.0,Unknown,20.00,105.50,Western Pacific
217925,NORA,1951,09/03/1951,3:00:00 PM,54.0,987.0,Unknown,19.92,105.40,Western Pacific
217926,NORA,1951,09/03/1951,6:00:00 PM,52.0,990.0,Unknown,20.00,105.00,Western Pacific
217927,NORA,1951,09/03/1951,9:00:00 PM,48.0,991.0,Unknown,20.40,103.95,Western Pacific


In [20]:
df[['Month', 'Day', 'year']] = df.date.str.split("/", expand=True)
df.drop(['date'], axis=1, inplace=True)

In [21]:
bins = [0, 1, 3, 7, 12, 18, 24, 31, 38, 46, 54, 63, 72, 250]
labels = [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]
# https://en.wikipedia.org/wiki/Beaufort_scale
df['beaufort_scale'] = pd.cut(df['wind_power'], bins, labels=labels)

In [22]:
df['Ocean'] = df['Ocean'].astype('category')
df['ocean_code'] = df['Ocean'].cat.codes

In [23]:
df.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 216596 entries, 0 to 217928
Data columns (total 13 columns):
 #   Column          Non-Null Count   Dtype   
---  ------          --------------   -----   
 0   storm_name      216596 non-null  object  
 1   year            216596 non-null  object  
 2   time            216596 non-null  object  
 3   wind_power      216596 non-null  float64 
 4   air_pressure    216596 non-null  float64 
 5   storm_type      216596 non-null  object  
 6   lat             216596 non-null  float64 
 7   long            216596 non-null  float64 
 8   Ocean           216596 non-null  category
 9   Month           216596 non-null  object  
 10  Day             216596 non-null  object  
 11  beaufort_scale  216596 non-null  category
 12  ocean_code      216596 non-null  int8    
dtypes: category(2), float64(4), int8(1), object(6)
memory usage: 18.8+ MB


In [24]:
df

,storm_name,year,time,wind_power,air_pressure,storm_type,lat,long,Ocean,Month,Day,beaufort_scale,ocean_code
0,ANA,2021,5:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.20,Atlantic Ocean,5,22,8,0
1,ANA,2021,8:00:00 AM,46.0,1006.0,Subtropical Storm,34.20,-62.50,Atlantic Ocean,5,22,8,0
2,ANA,2021,11:00:00 AM,46.0,1006.0,Subtropical Storm,34.30,-63.00,Atlantic Ocean,5,22,8,0
3,ANA,2021,5:00:00 PM,40.0,1007.0,Subtropical Storm,34.50,-62.40,Atlantic Ocean,5,22,8,0
4,ANA,2021,11:00:00 PM,46.0,1006.0,Subtropical Storm,35.00,-61.40,Atlantic Ocean,5,22,8,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
217924,NORA,1951,12:00:00 PM,58.0,985.0,Unknown,20.00,105.50,Western Pacific,09,03,10,5
217925,NORA,1951,3:00:00 PM,54.0,987.0,Unknown,19.92,105.40,Western Pacific,09,03,9,5
217926,NORA,1951,6:00:00 PM,52.0,990.0,Unknown,20.00,105.00,Western Pacific,09,03,9,5
217927,NORA,1951,9:00:00 PM,48.0,991.0,Unknown,20.40,103.95,Western Pacific,09,03,9,5


In [25]:
df.to_csv('cleaningDF.csv', index=False)
